# Librerias

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
from IPython.core.display import HTML
HTML("""<style>
    .output-plaintext, .output-stream, .output{
        white-space: pre !important;
        font-family: Monaco; # Any monospaced font should work
    }</style>""")

# Cargar data

In [0]:
def cargar_tabla(table_name):
    database_host_url = "bxbsgdghmeatencaj435-postgresql.services.clever-cloud.com"
    database_port = "5432"
    database_name = "bxbsgdghmeatencaj435"
    username = "ujixipqkhuwzgyrhdzkr"
    password = "RjyB6swJPse3F5ysdMpRokQWtqmcEd"

    table = (spark.read
      .format("postgresql")
      .option("dbtable", table_name)
      .option("host", database_host_url)
      .option("port", database_port)
      .option("database", database_name)
      .option("user", username)
      .option("password", password)
      .load()
    )
    return table

In [0]:
actor = cargar_tabla("actor")
film_actor = cargar_tabla("film_actor")
film = cargar_tabla("film")
film_category = cargar_tabla("film_category")
category = cargar_tabla("category")
language = cargar_tabla("language")
inventory = cargar_tabla("inventory")
rental = cargar_tabla("rental")
payment = cargar_tabla("payment")
customer = cargar_tabla("customer")
address = cargar_tabla("address")
city = cargar_tabla("city")
country = cargar_tabla("country")
staff = cargar_tabla("staff")
store = cargar_tabla("store")

In [0]:
type(film)

Out[5]: pyspark.sql.dataframe.DataFrame

In [0]:
print((film.count(), len(film.columns)))

(1000, 13)


In [0]:
film.show(5, True)

+-------+-----------------+--------------------+------------+-----------+---------------+-----------+------+----------------+------+--------------------+--------------------+--------------------+
|film_id|            title|         description|release_year|language_id|rental_duration|rental_rate|length|replacement_cost|rating|         last_update|    special_features|            fulltext|
+-------+-----------------+--------------------+------------+-----------+---------------+-----------+------+----------------+------+--------------------+--------------------+--------------------+
|    133|  Chamber Italian|A Fateful Reflect...|        2006|          1|              7|       4.99|   117|           14.99| NC-17|2013-05-26 14:50:...|          [Trailers]|'chamber':1 'fate...|
|    384| Grosse Wonderful|A Epic Drama of a...|        2006|          1|              5|       4.99|    49|           19.99|     R|2013-05-26 14:50:...| [Behind the Scenes]|'australia':18 'c...|
|      8|  Airport P

# Crear datalake

In [0]:
%sql

CREATE DATABASE IF NOT EXISTS landing LOCATION 'dbfs:///CTIC_PROJECT/landing';
CREATE DATABASE IF NOT EXISTS bronze LOCATION 'dbfs:///CTIC_PROJECT/bronze';
CREATE DATABASE IF NOT EXISTS silver LOCATION 'dbfs:///CTIC_PROJECT/silver';
CREATE DATABASE IF NOT EXISTS gold_ LOCATION 'dbfs:///CTIC_PROJECT/gold_';

In [0]:
%fs
ls dbfs:/CTIC_PROJECT

path,name,size,modificationTime
dbfs:/CTIC_PROJECT/bronze/,bronze/,0,0
dbfs:/CTIC_PROJECT/gold_/,gold_/,0,0
dbfs:/CTIC_PROJECT/landing/,landing/,0,0
dbfs:/CTIC_PROJECT/silver/,silver/,0,0


# Landing

In [0]:
actor.write.saveAsTable("landing.actor", mode="overwrite")
film_actor.write.saveAsTable("landing.film_actor", mode="overwrite")
film.write.saveAsTable("landing.film", mode="overwrite")
film_category.write.saveAsTable("landing.film_category", mode="overwrite")
category.write.saveAsTable("landing.category", mode="overwrite")
language.write.saveAsTable("landing.language", mode="overwrite")
inventory.write.saveAsTable("landing.inventory", mode="overwrite")
rental.write.saveAsTable("landing.rental", mode="overwrite")
payment.write.saveAsTable("landing.payment", mode="overwrite")
customer.write.saveAsTable("landing.customer", mode="overwrite")
address.write.saveAsTable("landing.address", mode="overwrite")
city.write.saveAsTable("landing.city", mode="overwrite")
country.write.saveAsTable("landing.country", mode="overwrite")
staff.write.saveAsTable("landing.staff", mode="overwrite")
store.write.saveAsTable("landing.store", mode="overwrite")

In [0]:
%fs
ls dbfs:/CTIC_PROJECT/landing

path,name,size,modificationTime
dbfs:/CTIC_PROJECT/landing/actor/,actor/,0,0
dbfs:/CTIC_PROJECT/landing/address/,address/,0,0
dbfs:/CTIC_PROJECT/landing/category/,category/,0,0
dbfs:/CTIC_PROJECT/landing/city/,city/,0,0
dbfs:/CTIC_PROJECT/landing/country/,country/,0,0
dbfs:/CTIC_PROJECT/landing/customer/,customer/,0,0
dbfs:/CTIC_PROJECT/landing/film/,film/,0,0
dbfs:/CTIC_PROJECT/landing/film_actor/,film_actor/,0,0
dbfs:/CTIC_PROJECT/landing/film_category/,film_category/,0,0
dbfs:/CTIC_PROJECT/landing/inventory/,inventory/,0,0


# Bronze

In [0]:
bronze_category = spark.sql("""
                            SELECT 
                                category_id,
                                name
                            FROM landing.category
                            WHERE category_id IN (15, 9, 8, 6, 2, 1, 13, 7, 14, 10, 3)
                            """)

bronze_category.write.saveAsTable("bronze.category", mode="overwrite")

In [0]:
bronze_film = spark.sql("""
                        SELECT 
                            film_id,
                            title,
                            description,
                            language_id,
                            rental_duration,
                            rental_rate,
                            rating                          
                        FROM landing.film
                        WHERE rating != 'G' AND rental_rate >= 1
                        """)

bronze_film.write.saveAsTable("bronze.film", mode="overwrite")

In [0]:
bronze_rental = spark.sql("""
                        SELECT 
                            rental_id,
                            DATE(rental_date) AS rental_date,
                            inventory_id,
                            customer_id,
                            DATE(return_date) AS return_date,
                            staff_id                            
                        FROM landing.rental
                        WHERE DATE(rental_date) >= '2005-07-05'
                        """)

bronze_rental.write.saveAsTable("bronze.rental", mode="overwrite")

In [0]:
bronze_customer = spark.sql("""
                        SELECT 
                            customer_id,
                            store_id,
                            first_name,
                            last_name,
                            address_id,
                            create_date
                        FROM landing.customer
                        WHERE active == 0
                        """)

bronze_customer.write.saveAsTable("bronze.customer", mode="overwrite")

In [0]:
bronze_country = spark.sql("""
                        SELECT
                            country_id,
                            country
                        FROM landing.country
                        WHERE country IN (
                                SELECT country
                                FROM (
                                        SELECT 
                                            co.country, COUNT(customer_id) AS cantidad
                                        FROM landing.customer AS c
                                        INNER JOIN landing.address AS a ON a.address_id = c.address_id
                                        INNER JOIN landing.city AS ci ON a.city_id = ci.city_id
                                        INNER JOIN landing.country AS co ON ci.country_id = co.country_id
                                        GROUP BY co.country) AS t
                                WHERE cantidad >= 5
                        )
                        """)

bronze_country.write.saveAsTable("bronze.country", mode="overwrite")

In [0]:
bronze_actor = spark.sql("""
                            SELECT 
                                actor_id,
                                first_name,
                                last_name
                            FROM landing.actor
                            """)

bronze_actor.write.saveAsTable("bronze.actor", mode="overwrite")

In [0]:
bronze_film_actor = spark.sql("""
                            SELECT 
                                actor_id,
                                film_id
                            FROM landing.film_actor
                            """)

bronze_film_actor.write.saveAsTable("bronze.film_actor", mode="overwrite")

In [0]:
bronze_film_category = spark.sql("""
                            SELECT 
                                film_id,
                                category_id
                            FROM landing.film_category
                            """)

bronze_film_category.write.saveAsTable("bronze.film_category", mode="overwrite")

In [0]:
bronze_language = spark.sql("""
                            SELECT 
                                language_id,
                                name
                            FROM landing.language
                            """)

bronze_language.write.saveAsTable("bronze.language", mode="overwrite")

In [0]:
bronze_inventory = spark.sql("""
                            SELECT 
                                inventory_id,
                                film_id,
                                store_id
                            FROM landing.inventory
                            """)

bronze_inventory.write.saveAsTable("bronze.inventory", mode="overwrite")

In [0]:
bronze_payment = spark.sql("""
                            SELECT 
                                payment_id,
                                customer_id,
                                staff_id,
                                rental_id,
                                amount,
                                DATE(payment_date) AS payment_date
                            FROM landing.payment
                            """)

bronze_payment.write.saveAsTable("bronze.payment", mode="overwrite")

In [0]:
bronze_address = spark.sql("""
                            SELECT 
                                address_id,
                                address,
                                district,
                                city_id,
                                postal_code
                            FROM landing.address
                            """)

bronze_address.write.saveAsTable("bronze.address", mode="overwrite")

In [0]:
bronze_city = spark.sql("""
                            SELECT 
                                city_id,
                                city,
                                country_id
                            FROM landing.city
                            """)

bronze_city.write.saveAsTable("bronze.city", mode="overwrite")

In [0]:
bronze_staff = spark.sql("""
                            SELECT 
                                staff_id,
                                first_name,
                                last_name,
                                address_id,
                                store_id                                
                            FROM landing.staff
                            WHERE active == True
                            """)

bronze_staff.write.saveAsTable("bronze.staff", mode="overwrite")

In [0]:
bronze_store = spark.sql("""
                            SELECT 
                                store_id,
                                manager_staff_id,
                                address_id
                            FROM landing.store
                            """)

bronze_store.write.saveAsTable("bronze.store", mode="overwrite")

In [0]:
%fs
ls dbfs:/CTIC_PROJECT/bronze

path,name,size,modificationTime
dbfs:/CTIC_PROJECT/bronze/actor/,actor/,0,0
dbfs:/CTIC_PROJECT/bronze/address/,address/,0,0
dbfs:/CTIC_PROJECT/bronze/category/,category/,0,0
dbfs:/CTIC_PROJECT/bronze/city/,city/,0,0
dbfs:/CTIC_PROJECT/bronze/country/,country/,0,0
dbfs:/CTIC_PROJECT/bronze/customer/,customer/,0,0
dbfs:/CTIC_PROJECT/bronze/film/,film/,0,0
dbfs:/CTIC_PROJECT/bronze/film_actor/,film_actor/,0,0
dbfs:/CTIC_PROJECT/bronze/film_category/,film_category/,0,0
dbfs:/CTIC_PROJECT/bronze/inventory/,inventory/,0,0


# Silver

In [0]:
dim_film = spark.sql("""
                SELECT 
                    f.film_id,
                    f.title,
                    f.description,
                    f.rental_duration,
                    f.rental_rate,
                    f.rating,
                    c.name AS category,
                    l.name AS language,
                    CONCAT(a.first_name,' ', a.last_name) AS actor
                FROM bronze.film AS f
                INNER JOIN bronze.language AS l ON f.language_id = l.language_id
                INNER JOIN bronze.film_actor AS fa ON f.film_id = fa.film_id
                INNER JOIN bronze.actor AS a ON fa.actor_id = a.actor_id
                INNER JOIN bronze.film_category AS fc ON f.film_id = fc.film_id
                INNER JOIN bronze.category AS c ON fc.category_id = c.category_id
                """)

dim_film.write.saveAsTable("silver.dim_film", mode="overwrite")

In [0]:
dim_customer = spark.sql("""
                SELECT 
                    c.customer_id,
                    CONCAT(c.first_name,' ', c.last_name) AS customer,
                    c.create_date,
                    a.address,
                    a.district,
                    a.postal_code,
                    ci.city,
                    co.country           
                FROM bronze.customer AS c
                INNER JOIN bronze.address AS a ON a.address_id = c.address_id
                INNER JOIN bronze.city AS ci ON a.city_id = ci.city_id
                INNER JOIN bronze.country AS co ON ci.country_id = co.country_id
                """)

dim_customer.write.saveAsTable("silver.dim_customer", mode="overwrite")

In [0]:
dim_staff = spark.sql("""
            SELECT 
                sta.staff_id,
                CONCAT(sta.first_name,' ', sta.last_name) AS staff,
                sto.manager_staff_id,
                a.address,
                a.district,
                a.postal_code,
                ci.city,
                co.country
            FROM bronze.staff AS sta
            INNER JOIN bronze.store AS sto ON sta.store_id = sto.store_id
            INNER JOIN bronze.address AS a ON a.address_id = sto.address_id
            INNER JOIN bronze.city AS ci ON a.city_id = ci.city_id
            INNER JOIN bronze.country AS co ON ci.country_id = co.country_id
            """)

dim_staff.write.saveAsTable("silver.dim_staff", mode="overwrite")

In [0]:
dim_store = spark.sql("""
            SELECT
                    sto.store_id,
                    sto.manager_staff_id,
                    a.address,
                    a.district,
                    a.postal_code,
                    ci.city,
                    co.country
                FROM bronze.store AS sto
                INNER JOIN bronze.address AS a ON a.address_id = sto.address_id
                INNER JOIN bronze.city AS ci ON a.city_id = ci.city_id
                INNER JOIN bronze.country AS co ON ci.country_id = co.country_id
            """)

dim_store.write.saveAsTable("silver.dim_store", mode="overwrite")

In [0]:
fact_rental_inventory = spark.sql("""
                        SELECT
                            r.rental_id,
                            r.rental_date,
                            r.inventory_id,
                            r.customer_id,
                            r.return_date,
                            r.staff_id,
                            i.film_id,
                            i.store_id
                        FROM bronze.rental AS r
                        INNER JOIN bronze.inventory AS i ON r.inventory_id = i.inventory_id
                        """)

fact_rental_inventory.write.saveAsTable("silver.fact_rental_inventory", mode="overwrite")

In [0]:
fact_payment = spark.sql("""
                SELECT 
                    *
                FROM bronze.payment
                """)

fact_payment.write.saveAsTable("silver.fact_payment", mode="overwrite")

In [0]:
%fs
ls dbfs:/CTIC_PROJECT/silver

path,name,size,modificationTime
dbfs:/CTIC_PROJECT/silver/dim_customer/,dim_customer/,0,0
dbfs:/CTIC_PROJECT/silver/dim_film/,dim_film/,0,0
dbfs:/CTIC_PROJECT/silver/dim_staff/,dim_staff/,0,0
dbfs:/CTIC_PROJECT/silver/dim_store/,dim_store/,0,0
dbfs:/CTIC_PROJECT/silver/fact_payment/,fact_payment/,0,0
dbfs:/CTIC_PROJECT/silver/fact_rental_inventory/,fact_rental_inventory/,0,0


# Gold

In [0]:
reporte_rental = spark.sql("""
                SELECT
                        ri.rental_id,
                        f.title,
                        f.description,
                        f.rental_duration,
                        f.rental_rate,
                        f.rating,
                        f.category,
                        f.language,
                        f.actor,
                        s.address AS store_address,
                        s.district AS store_district,
                        s.postal_code AS store_postal_code,
                        s.city AS store_city,
                        s.country AS store_country,
                        c.customer,
                        c.create_date AS customer_create_date,
                        c.address AS customer_address,
                        c.district AS customer_district,
                        c.postal_code AS customer_postal_code,
                        c.city AS customer_city,
                        c.country AS customer_country,
                        st.staff,
                        st.address AS staff_address,
                        st.district AS staff_district,
                        st.postal_code AS staff_postal_code,
                        st.city AS staff_city,
                        st.country AS staff_country
                    FROM silver.fact_rental_inventory AS ri
                    INNER JOIN silver.dim_film AS f ON ri.film_id = f.film_id
                    INNER JOIN silver.dim_store AS s ON ri.store_id = s.store_id
                    INNER JOIN silver.dim_customer AS c ON ri.customer_id = c.customer_id
                    INNER JOIN silver.dim_staff AS st ON ri.staff_id = st.staff_id
                """)

reporte_rental.write.saveAsTable("gold_.reporte_rental", mode="overwrite")

In [0]:
%fs
ls dbfs:/CTIC_PROJECT/gold_

path,name,size,modificationTime
dbfs:/CTIC_PROJECT/gold_/reporte_rental/,reporte_rental/,0,0
